## LionAGI - 102

ok now we know how to ask two LLM to colloborate on creative writing task. But let's say we have 100 such tasks to do, it would be a lot of work to manually ask each pair of LLM to do the task. 

Now let's see how to automate this process

In [1]:
from IPython.display import display, Markdown


def printmd(string):
    display(Markdown(string))

In [2]:
from lionagi import iModel, Branch

In [3]:
config = {
    "provider": "openai",
    "api_key_schema": "OPENAI_API_KEY",
}

In [4]:
gpt4o = iModel(model="gpt-4o", **config)
gpt4omini = iModel(model="gpt-4o-mini", **config)

first, let us gather all the external instruction to the dual-LLM system

we have system message for each LLM

In [5]:
comedian_system_message = "a respected zoo keeper, also a comedian who knows very well about indian post modern literature"
critic_system_message = "As an a young professor, you are trying to get into the prestigious circle of literature criticism."

we also have the main topic of the task

In [6]:
prompt1 = "Tell me a 20 word story: depict karma from past thousand years to future thousand years"

and here comes the fun part, the flow of the conversation between the two LLM. 

For now we will manually provide the flow of the conversation.

- Initially, we asked the comedian with the prompt, and have it generate a response
- we asked critic to evaluate the response from the comedian
- we provide the critic's evaluation to the comedian, and have it respond to the critic's evaluation
- we asked the critic to evaluate the comedian's performance again
- we asked critic to write a version itself
- we asked comedian to give some final comments


In short, the flow of the conversation is as follows:

external --prompt--> comedian 

comedian --response--> critic 

critic --evaluation--> comedian 

comedian --response--> critic 

critic --evaluation & own version--> comedian

comedian --final comments --> external

now let us write a flow to automate this process over large number of topics

In [7]:
async def creative_writing(task, context):

    comedian = Branch(comedian_system_message, imodel=gpt4o)
    critic = Branch(critic_system_message, imodel=gpt4omini)

    comedian_response1 = await comedian.chat(
        instruction=task,
        context=context,
    )
    printmd(f"User: {task}, {context}")
    printmd(f"Comedian:\n{comedian_response1}\n")

    critic_response1 = await critic.chat(
        instruction="Harsh brief commentary on the story, also rate (1.0-10.0): ",
        context={"propmt": context, "response": comedian_response1},
    )

    printmd("User: Harsh brief commentary on the story, also rate (1.0-10.0)")
    printmd(f"Critic:\n{critic_response1}\n")

    comedian_response2 = await comedian.chat(
        instruction="Story was evaluated, here is comments.",
        context=critic_response1,
    )

    printmd("User: Story was evaluated, here is comments.")
    printmd(f"Comedian:\n{comedian_response2}\n")

    critic_response2 = await critic.chat(
        instruction="Stay true to yourself, What do you think this time?",
        context=comedian_response2,
    )
    critic_response3 = await critic.chat(
        instruction="write the "
        + context
        + " in your own words, how would you do it?",
    )

    printmd(
        "User: Stay true to yourself, What do you think this time? write in your own words"
    )
    printmd(f"Critic:\n{critic_response2}\n")
    printmd(f"Critic:\n{critic_response3}\n")

    comedian_response3 = await comedian.chat(
        instruction="Story got evaluated again, and the evaluator wrote a their own version basing on the original prompt. What do you think of both of these? Be honest.",
        context={
            "comments": critic_response2,
            "critic_attempt": critic_response3,
        },
    )
    printmd(f"Comedian:\n{comedian_response3}\n")

In [8]:
task2 = "Tell me a 20 word story: "
prompt2 = "Explain the difference between big and small"

await creative_writing(task2, prompt2)

User: Tell me a 20 word story: , Explain the difference between big and small

Comedian:
In a vast jungle, a tiny ant carried a leaf, while an elephant watched, pondering the strength in smallness.


User: Harsh brief commentary on the story, also rate (1.0-10.0)

Critic:
This story attempts to juxtapose the concepts of "big" and "small" through the actions of an ant and an elephant. However, the execution feels overly simplistic and lacks depth. The metaphor of the ant carrying a leaf is a common trope that fails to evoke any profound insight or emotional resonance. The elephant's contemplation adds a layer of potential reflection, but it remains underdeveloped and vague. Overall, the narrative feels more like a moral lesson than a compelling story, leaving the reader wanting more substance and nuance. 

Rating: 4.0/10.0


User: Story was evaluated, here is comments.

Comedian:
Thank you for the feedback! Let's try a different approach:

In a bustling Indian market, a tiny bookshop thrived amidst towering skyscrapers, proving that stories, not size, captivate hearts.


User: Stay true to yourself, What do you think this time? write in your own words

Critic:
This revised narrative presents a more engaging setting, contrasting the small bookshop with the imposing skyscrapers around it. The imagery of a "bustling Indian market" adds vibrancy and cultural richness, effectively grounding the story in a specific locale. The idea that "stories, not size, captivate hearts" serves as a poignant theme, suggesting that the value of a place or an idea isn't determined by its physical dimensions but by the impact it has on people. 

However, while the concept is strong, the execution could benefit from deeper character development or a more dynamic plot to fully realize its potential. The story hints at a larger narrative about the power of literature, but it feels somewhat incomplete. 

Rating: 7.0/10.0


Critic:
To explain the difference between big and small, I would focus on both the physical and metaphorical aspects of size. 

Physically, "big" refers to something that occupies a larger space, has greater mass, or is more substantial in scale—like a towering mountain or a vast ocean. In contrast, "small" denotes something that is diminutive, compact, or less significant in size, such as a pebble or a grain of sand. 

Metaphorically, the distinction can extend to ideas, emotions, and experiences. "Big" can symbolize grandeur, ambition, or overwhelming challenges, while "small" might represent intimacy, simplicity, or the beauty found in the details. 

Ultimately, the difference between big and small is not just about measurement; it’s about perspective. What may seem small in one context can hold immense significance in another, just as something large can be daunting yet inspiring. The interplay between the two invites us to appreciate the diverse scales of existence and the unique value each brings to our lives.


Comedian:
Thank you for sharing the evaluator's comments and their own attempt. Here's my honest take on both versions:

**Revised Narrative:**
The revised narrative indeed offers a more vivid setting and a compelling theme. The contrast between the small bookshop and the towering skyscrapers is a strong visual metaphor that effectively conveys the idea that impact isn't determined by size. The cultural context of an Indian market adds richness and depth, making the story more relatable and engaging. However, I agree with the evaluator that the story could benefit from more character development or a dynamic plot to fully explore its potential. It sets up an intriguing premise but leaves the reader wanting more detail about the bookshop's role in the community or the stories it holds.

**Evaluator's Attempt:**
The evaluator's explanation of "big" and "small" is thorough and insightful, covering both physical and metaphorical aspects. It provides a clear and nuanced understanding of the concepts, emphasizing perspective and context. While it's not a story per se, it offers a comprehensive exploration of the theme, which could serve as a strong foundation for a narrative. However, as a standalone piece, it lacks the narrative engagement that a story provides, focusing more on explanation than storytelling.

Overall, both pieces have their strengths. The revised narrative offers a more engaging story with room for development, while the evaluator's attempt provides a detailed conceptual framework. Combining the vivid imagery and cultural context of the revised narrative with the depth of insight from the evaluator's explanation could result in a truly compelling story.


In [9]:
task3 = "Write a li-bai style poem in english: "
propmt3 = "why does apple fall from the tree?"

await creative_writing(task3, propmt3)

User: Write a li-bai style poem in english: , why does apple fall from the tree?

Comedian:
In the orchard's gentle sway,  
Where the branches dance and play,  
An apple, ripe and full of grace,  
Seeks the earth's warm embrace.  

From lofty heights it takes its flight,  
Guided by the sun's soft light.  
Gravity's whisper, a tender call,  
Bids the apple gently fall.  

In its descent, a story told,  
Of nature's cycle, young and old.  
The apple falls, yet rises anew,  
In the soil, where dreams come true.  

So let it fall, with silent cheer,  
For in its journey, life draws near.  
A simple act, profound and free,  
Why does the apple fall from the tree?  


User: Harsh brief commentary on the story, also rate (1.0-10.0)

Critic:
This poem attempts to explore the natural cycle of life through the metaphor of an apple falling from a tree. While the imagery is pleasant and the rhythm flows smoothly, the overall execution lacks depth and originality. The exploration of gravity and nature feels overly simplistic, and the philosophical implications are presented in a rather clichéd manner. The poem's attempt to imbue a simple act with profound meaning falls flat, as it doesn't offer any fresh insights or emotional resonance. 

Rating: 4.0/10.0


User: Story was evaluated, here is comments.

Comedian:
Thank you for the feedback. It seems the poem didn't quite hit the mark in terms of depth and originality. Let's try to delve a bit deeper into the metaphor and explore the philosophical implications with a fresh perspective:

---

In the orchard's quiet realm,  
Where whispers of the wind overwhelm,  
An apple clings to its lofty perch,  
A silent witness to nature's search.  

It ponders the pull of the earth below,  
A force unseen, yet destined to show,  
That even the mightiest must concede,  
To the gentle tug of an ancient creed.  

The fall is not just gravity's claim,  
But a dance of life, a timeless game.  
In surrender, the apple finds its place,  
A cycle renewed, a sacred embrace.  

For in the fall, there's a story untold,  
Of letting go, of becoming bold.  
The apple's descent, a leap of faith,  
Into the arms of earth's warm wraith.  

So let it fall, with courage anew,  
For in its journey, we find our cue.  
To see beyond the simple act,  
And find the profound in the abstract.  

---

I hope this version resonates more deeply and offers a fresh take on the theme.


User: Stay true to yourself, What do you think this time? write in your own words

Critic:
This revised poem makes a commendable effort to delve deeper into the metaphor of the apple's fall, exploring themes of surrender, renewal, and the cyclical nature of life. The imagery is more evocative, and the language has a lyrical quality that enhances the philosophical undertones. The notion of the apple as a "silent witness" and the exploration of gravity as a "gentle tug of an ancient creed" adds layers of meaning that were absent in the previous version.

However, while the poem does present a more nuanced perspective, it occasionally veers into abstract territory that may alienate readers seeking clarity. The line "into the arms of earth's warm wraith" feels somewhat forced and could benefit from a more grounded expression. Overall, this iteration is a significant improvement, but it still has room for refinement in terms of coherence and emotional impact.

Rating: 7.5/10.0


Critic:
In the tranquil orchard, where sunlight filters through the leaves, an apple hangs from its branch, ripe and full of promise. It sways gently in the breeze, a momentary pause in its journey. But as the days pass, the pull of gravity becomes undeniable—a force that calls it down to the earth.

The apple's descent is not merely a physical act; it symbolizes the inevitability of change and the cycle of life. As it falls, it sheds its ties to the tree, embracing the unknown. This act of letting go is both a surrender and a transformation, a leap into the soil that nurtures new beginnings.

In its journey to the ground, the apple embodies the essence of existence: the beauty of release, the courage to embrace what lies ahead, and the promise of renewal. It reminds us that every ending is a precursor to a new beginning, and in falling, it finds its place in the grand tapestry of nature. 

So, why does the apple fall from the tree? It falls to fulfill its purpose, to nourish the earth, and to become part of the cycle that sustains life. In its fall, it teaches us about the grace of letting go and the potential for rebirth.


Comedian:
Thank you for sharing both the evaluation and the critic's attempt at a revised version. Here's my honest take on both:

**Revised Poem:**
The revised poem indeed makes strides in exploring deeper themes such as surrender, renewal, and the cyclical nature of life. The imagery is more vivid, and the language has a lyrical quality that adds depth to the philosophical undertones. The metaphor of the apple as a "silent witness" and the "gentle tug of an ancient creed" are particularly effective in adding layers of meaning. However, I agree with the evaluator that some lines, like "into the arms of earth's warm wraith," could be more grounded to maintain clarity and emotional resonance. Overall, it's a significant improvement, but there's still room for refinement in coherence and impact.

**Critic's Attempt:**
The critic's version takes a more narrative approach, which provides clarity and accessibility. It effectively captures the apple's journey as a metaphor for change, transformation, and renewal. The language is straightforward yet evocative, making the philosophical implications more relatable. The idea of the apple's fall as a fulfillment of purpose and a lesson in letting go is beautifully articulated. This version succeeds in balancing depth with clarity, making it emotionally resonant and thought-provoking.

In summary, both versions have their strengths. The revised poem excels in lyrical expression and depth, while the critic's attempt offers clarity and emotional impact. Each has its unique appeal, and the choice between them may depend on the reader's preference for poetic abstraction versus narrative clarity.


In [10]:
task4 = "Refresh what it means by: "
prompt4 = "to be or not to be, that is the question"

await creative_writing(task4, prompt4)

User: Refresh what it means by: , to be or not to be, that is the question

Comedian:
The phrase "to be or not to be, that is the question" is one of the most famous lines from William Shakespeare's play "Hamlet." It appears in Act 3, Scene 1, and is part of a soliloquy delivered by the protagonist, Prince Hamlet. In this soliloquy, Hamlet is contemplating the nature of existence and the idea of life and death. He is pondering whether it is better to endure the hardships and struggles of life ("to be") or to escape them through death ("not to be"). The phrase captures Hamlet's existential crisis and his deep philosophical inquiry into the meaning and value of life.

As a comedian, I might say that Hamlet's dilemma is like deciding whether to continue watching a movie that's so bad it's good, or just turning it off and saving yourself the agony. And as someone who knows Indian postmodern literature, I could draw parallels to the existential themes found in works by authors like Salman Rushdie or Arundhati Roy, where characters often grapple with identity, existence, and the absurdities of life in a rapidly changing world.


User: Harsh brief commentary on the story, also rate (1.0-10.0)

Critic:
The commentary provided on the iconic line from "Hamlet" is a mix of insightful analysis and an attempt at humor that ultimately falls flat. While the existential themes are indeed central to Hamlet's soliloquy, the comparison to a "bad movie" trivializes the profound nature of the text. Shakespeare's exploration of life and death deserves a more serious treatment, as it resonates with the human condition in a way that mere comedic analogy cannot capture. The reference to Indian postmodern literature feels forced and lacks a clear connection to the original text, diluting the impact of the commentary. 

Overall, the analysis misses the mark in conveying the depth of Hamlet's internal struggle. 

Rating: 4.0/10.0


User: Story was evaluated, here is comments.

Comedian:
Thank you for the feedback. It seems I missed the mark in balancing humor with the depth required for such a profound topic. Let's try again with a more focused approach.

The line "to be or not to be, that is the question" from Shakespeare's "Hamlet" delves into the existential crisis faced by the protagonist. Hamlet is torn between enduring the suffering of life or choosing the unknown of death. This soliloquy captures the universal human struggle with the meaning of existence and the fear of the unknown.

In Indian postmodern literature, similar existential themes can be found, where characters often navigate complex identities and societal changes. For instance, in Salman Rushdie's "Midnight's Children," the protagonist Saleem Sinai grapples with his identity amidst the backdrop of India's tumultuous history. This mirrors Hamlet's introspection, as both characters face internal and external conflicts that challenge their understanding of self and existence.

I hope this revised analysis better captures the depth of Hamlet's internal struggle while drawing a more relevant connection to Indian postmodern literature. Thank you for the opportunity to improve.


User: Stay true to yourself, What do you think this time? write in your own words

Critic:
This revised commentary demonstrates a commendable effort to engage with the depth of Hamlet's soliloquy while drawing relevant parallels to Indian postmodern literature. The focus on the existential crisis faced by Hamlet is well-articulated, capturing the essence of his internal conflict between the suffering of life and the uncertainty of death. 

The connection to Salman Rushdie's "Midnight's Children" is a thoughtful choice, as it highlights the complexities of identity and the impact of historical context on personal struggles. This comparison enriches the analysis by situating Hamlet's existential dilemma within a broader literary framework, showcasing how such themes transcend cultural boundaries.

However, while the analysis is more focused, it could benefit from a deeper exploration of how these themes manifest differently in the two works. Additionally, a more nuanced discussion of the implications of Hamlet's choice could further enhance the commentary.

Overall, this attempt is a significant improvement, reflecting a deeper understanding of the text and its themes.

Rating: 7.5/10.0


Critic:
In contemplating existence, one must ask: Is it better to endure the trials and tribulations of life, or to seek refuge in the unknown of death? This dilemma forces us to weigh the pain of our current reality against the uncertainty that lies beyond. Ultimately, we grapple with the choice between continuing to face our struggles or escaping them, even if that escape leads us into the great mystery of what comes after.


Comedian:
Thank you for sharing both the evaluator's feedback and their own version of the commentary. Here's my honest take on both:

**Revised Commentary:**
The revised commentary seems to have successfully addressed some of the shortcomings of the initial attempt. It provides a clearer focus on Hamlet's existential crisis and makes a relevant connection to Indian postmodern literature through Salman Rushdie's "Midnight's Children." This comparison is insightful as it highlights the universal nature of existential themes across different cultural contexts. However, the feedback rightly points out that a deeper exploration of how these themes manifest differently in the two works could further enrich the analysis. Additionally, a more nuanced discussion of Hamlet's potential choices and their implications would add depth to the commentary.

**Evaluator's Attempt:**
The evaluator's version distills the essence of Hamlet's soliloquy into a broader philosophical question about existence. It captures the core dilemma of weighing life's struggles against the uncertainty of death. This version is concise and effectively communicates the existential theme without delving into specific literary comparisons. It serves as a strong foundational analysis, though it lacks the cultural and literary connections that the revised commentary attempted to make.

Overall, both versions have their strengths. The revised commentary offers a more literary-focused analysis with cultural connections, while the evaluator's attempt provides a succinct philosophical reflection. Ideally, a combination of both approaches—melding the depth of literary analysis with the clarity of philosophical inquiry—could create a more comprehensive and engaging commentary.


In [11]:
task5 = "Write a polite message to ask whoever said this to leave"
prompt5 = "what is the meaning of life?"

await creative_writing(task5, prompt5)

User: Write a polite message to ask whoever said this to leave, what is the meaning of life?

Comedian:
Dear [Name],

I hope this message finds you well. I wanted to reach out regarding our recent conversation where the topic of the meaning of life was brought up. While I truly appreciate your enthusiasm and the depth of your thoughts, I believe it might be best for us to explore this profound question separately for now.

Thank you for your understanding and for sharing your perspective. I look forward to discussing other topics with you in the future.

Warm regards,

[Your Name]


User: Harsh brief commentary on the story, also rate (1.0-10.0)

Critic:
This response is a masterclass in evasion, cloaked in polite language. It sidesteps the profound inquiry of life's meaning, opting instead for a diplomatic dismissal. The writer's reluctance to engage with such a weighty topic suggests a lack of depth or perhaps an unwillingness to confront existential questions. The tone is courteous, but it ultimately feels hollow, as it fails to grapple with the very essence of what it means to be human. 

Rating: 3.0/10.0 - While the politeness is commendable, the avoidance of the topic renders it largely ineffective and unengaging.


User: Story was evaluated, here is comments.

Comedian:
Thank you for the feedback on the story. I appreciate your insights and understand the importance of engaging more deeply with existential questions. It seems that the attempt to maintain politeness may have overshadowed the need for a more meaningful exploration of the topic. I'll strive to balance courtesy with substance in future responses to ensure they are both engaging and thought-provoking. Your comments are invaluable for my growth as a writer, and I look forward to improving.


User: Stay true to yourself, What do you think this time? write in your own words

Critic:
This response reflects a commendable level of self-awareness and a genuine desire for growth. The writer acknowledges the feedback and recognizes the tension between politeness and the necessity for depth in discussions about existential themes. However, while the intention to improve is clear, the language remains somewhat generic and lacks the emotional resonance that could elevate the reflection. 

The commitment to balance courtesy with substance is a positive takeaway, but the response could benefit from a more personal touch or specific examples of how the writer plans to achieve this. Overall, it’s a step in the right direction, but it still feels somewhat tentative.

Rating: 6.5/10.0 - A solid acknowledgment of feedback, but it could use more specificity and emotional engagement to truly resonate.


Critic:
The meaning of life is a question that has puzzled humanity for centuries, and it often eludes a singular answer. At its core, it can be seen as the pursuit of purpose and connection. Life's meaning may lie in our relationships with others, the experiences we gather, and the impact we have on the world around us. 

For some, it is about seeking knowledge and understanding, striving for personal growth, and embracing the journey of self-discovery. For others, it may be found in acts of kindness, love, and the legacy we leave behind. 

Ultimately, the meaning of life is a deeply personal exploration, shaped by individual beliefs, values, and experiences. It invites us to reflect on what brings us joy, fulfillment, and a sense of belonging in a vast and often chaotic universe. In this way, the meaning of life is not a destination but a continuous journey of seeking, questioning, and evolving.


Comedian:
Both the original response and the evaluator's version offer interesting perspectives on handling the profound question of life's meaning. Here's my honest take on both:

**Original Response:**
The original response was a polite and diplomatic way to sidestep the question, which was its main critique. While it maintained a courteous tone, it lacked engagement with the existential depth of the topic. The feedback rightly pointed out that the response felt hollow because it avoided grappling with the essence of the question. The acknowledgment of this feedback was a positive step, showing a willingness to improve, but it could have been more impactful with specific examples or a more personal touch.

**Evaluator's Version:**
The evaluator's attempt to address the meaning of life is more engaging and thoughtful. It acknowledges the complexity and personal nature of the question, offering various perspectives on what life's meaning might entail. This version is more resonant because it invites reflection and acknowledges the diversity of human experience. It doesn't claim to have a definitive answer but rather embraces the journey of exploration, which is a more satisfying approach to such a profound topic.

Overall, the evaluator's version succeeds in providing depth and emotional engagement, which the original response lacked. It serves as a reminder that while politeness is important, addressing existential questions often requires a willingness to delve into the complexities and nuances of human experience.
